# Bibliotecas importadas

In [ ]:
import re
import requests
import pandas as pd
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from seleniumbase import Driver
from seleniumbase import page_actions

# Tornando o Webdriver Indetectável

In [66]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = uc.Chrome(use_subprocess=True,options=options)

Acessando o site Research Gate

In [67]:
driver.get('https://www.researchgate.net')
autor = input()

Enviando Input informando o autor e acessando seção de autores

In [68]:
element = driver.find_element(By.CLASS_NAME, 'index-search-field__input')
element.send_keys(autor)
element = driver.find_element(By.TAG_NAME, 'button')
element.click()
element = driver.find_element(By.LINK_TEXT, 'Authors')
element.click()
element = driver.find_elements(By.CLASS_NAME, 'nova-legacy-v-person-item')
for ele in element:
    condicao = [(1,i)  if i in autor.split() else (i,0) for i in ele.text.split('\n')][0]
    tamanho = len(condicao[0].split())
    if tamanho >= len(autor.split()):
        elemento = ele.find_elements(By.TAG_NAME, 'a')
        elemento.click()

Utilizando Beautiful Soup para raspar os links com a Tag `Article`

In [74]:
current_html = driver.page_source
soup = BeautifulSoup(current_html, 'html.parser')

results = soup.find("div", {"class" : "nova-legacy-c-card nova-legacy-c-card--spacing-xl nova-legacy-c-card--elevation-1-above"})

padrao_artigo = re.compile('[a-z]{11}\/[0-9]{9,10}')
link = []

for i in results:
    for div in i.find_all('a', {'class':'nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-bare'}):
        if len(re.findall(padrao_artigo, str(div.get('href')))) == 1:
            link.append(div.get('href'))

Construindo a tabela excel com os links de artigos

In [75]:
data = {'Título': [i[51:] for i in link],
       'Links': link,
       }
dados = pd.DataFrame(data)
dados.to_excel(f'artigos_{autor}_research_gate.xlsx')

# Observações

Esse projeto foi bem divertido de fazer. Comecei tentando ele no colab, mas infelizmente tive problema com captcha do site `Research Gate`.Por esse motivo, preferi escrevê-lo no Jupyter Notebook. Nesse notebook ficou com menos empecilhos para alcançar seu objetivo, pois aqui consegui tornar, depois de algumas pesquisas, tornar o `WebDriver` indetectável, segundo a biblioteca importada `undetected_chromedriver`. Isso foi uma descoberta feliz, pois estava tendo muitos problemas com a detecção de robôs dos sites. Enfim, espero que gostem e se quiserem colaborar ou fazer críticas ou dar dicas de biblioteca de raspagem de dados, meu e-mail é: dimitri.limaf@outlook.com